In [1]:
%%writefile benchmark.py
"""This script is written from benchmark.ipynb. Do not edit directly."""

# Disable tensorflow spam (needs to happen before tensorflow gets imported)
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # set to "2" to see TensorRT errors
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("tensorflow").addHandler(logging.NullHandler(logging.ERROR))

import argparse
import numpy as np
import pandas as pd
import tensorflow as tf
import system
from video import read_frames
from trtutils import OptimizedModel
from time import perf_counter


system.disable_preallocation()


def log_result(data, filename="benchmark.csv"):
    """Append a dictionary of scalar keys as a row to a CSV file."""
#     data = pd.DataFrame({k: [v] for k, v in data.items()})
    if os.path.exists(filename):
        data = pd.concat([
            pd.read_csv(filename),
            data,
        ])
    data.to_csv(filename, index=False)


def benchmark(model, precision, test_data_path, n_frames, batch_size, reps, grayscale):
    trt_model_path = f"{model}/trtmodel_FP{precision}"

    imgs = read_frames(test_data_path, np.arange(n_frames), grayscale=grayscale)
    trt_model = OptimizedModel(saved_model_dir=trt_model_path)

    res = None
    dts = []
    for rep in range(reps + 1):
        for i in range(0, n_frames, batch_size):
            t0 = perf_counter()
            preds = trt_model.predict(imgs[i:(i+batch_size)], numpy=True)
            dt = perf_counter() - t0
            if rep > 0:
                dts.append(dt)
    dts = np.array(dts)

    res_ = pd.DataFrame({"batch_size": np.full(dts.shape, batch_size), "dts": dts})
    if res is None:
        res = res_
    else:
        res = pd.concat([res, res_])

    res["fps"] = res["batch_size"] / res["dts"]
    res["model"] = model
    res["precision"] = precision
    res["trt_model_path"] = trt_model_path
    res["test_data_path"] = test_data_path
    res["n_frames"], res["img_height"], res["img_width"] = imgs.shape[:-1]
    
    log_result(res, filename="trt_benchmarks.csv")
    
    print(res.groupby("batch_size")[["fps", "dts"]].agg(["mean", "std"]).to_string())
    

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("model", type=str)
    parser.add_argument("precision", type=str)
    parser.add_argument("test_data", type=str)
    parser.add_argument("--n_frames", type=int, default=1280)
    parser.add_argument("--batch_size", type=int, default=32)
    parser.add_argument("--reps", type=int, default=5)
    parser.add_argument("--grayscale", dest="grayscale", action="store_true")
    parser.add_argument("--no-grayscale", dest="grayscale", action="store_false")
    parser.set_defaults(grayscale=True)
    args = parser.parse_args()

    benchmark(args.model, args.precision, args.test_data, args.n_frames, args.batch_size, args.reps, args.grayscale)

Overwriting benchmark.py


In [2]:
import pandas as pd

mice_of_centroid = "sleap-data/datasets/wang_4mice_john/labels.full/models.random_split1/centroid.210507_132508.centroid.n=900"
mice_of_td = "sleap-data/experiments/best_model_replicates/mice_of_td.210509_231024.centered_instance.n=900"
mice_of_clips = """
sleap-data/datasets/wang_4mice_john/clips/OFTephys-0055-08@14616-18736.mp4
sleap-data/datasets/wang_4mice_john/clips/OFTsocialgroup-0000-00@117000-121700.mp4
sleap-data/datasets/wang_4mice_john/clips/OFTsocial5mice-0000-00.mp4.old.predictions@3700-6260.mp4
""".strip().split()


flies13_centroid = "sleap-data/datasets/wt_gold.13pt/models.tracking_split2/centroid.fast.210504_182918.centroid.n=1800"
flies13_td = "sleap-data/datasets/wt_gold.13pt/models.tracking_split2/td_fast.210505_012601.centered_instance.n=1800"
flies13_td_id = "sleap-data/datasets/wt_gold.13pt/models.tracking_split2/td_id.fast.v2.210519_111253.multi_class_topdown.n=1800"
flies13_clips = """
sleap-data/datasets/wt_gold.13pt/clips/single_fly@100000-110000.mp4
sleap-data/datasets/wt_gold.13pt/clips/190719_090330_wt_18159206_rig1.2@15000-17560.mp4
sleap-data/datasets/wt_gold.13pt/clips/three_flies@8800-16800.mp4
sleap-data/datasets/wt_gold.13pt/clips/four_flies@1000-11000.mp4
sleap-data/datasets/wt_gold.13pt/clips/eight_flies@180000-200000.mp4
""".strip().split()


fly32_single = "sleap-data/datasets/BermanFlies/models.random_split1/single.fast_unet32.210524_171130.single_instance.n=1350"
fly32_clips = ["sleap-data/datasets/BermanFlies/clips/072212_163153@10000-13200.mp4"]


gerbils_centroid = "sleap-data/datasets/nyu-gerbils/cohort1_compressedTalmo_23vids_march_7_to_march_17/models.random_split1.day001/centroid.210504_225945.centroid.n=383"
gerbils_td_id = "sleap-data/datasets/nyu-gerbils/cohort1_compressedTalmo_23vids_march_7_to_march_17/models.random_split1.day001/td_id.210505_002058.multi_class_topdown.n=383"
gerbils_clips = ["sleap-data/datasets/nyu-gerbils/clips/2020-3-10_daytime_5mins_compressedTalmo@3200-5760.mp4"]


test_data_info = pd.DataFrame([
    {"test_data_path": "sleap-data/datasets/wang_4mice_john/clips/OFTephys-0055-08@14616-18736.mp4", "dataset": "mice_of", "n_instances": 2},
    {"test_data_path": "sleap-data/datasets/wang_4mice_john/clips/OFTsocialgroup-0000-00@117000-121700.mp4", "dataset": "mice_of", "n_instances": 4},
    {"test_data_path": "sleap-data/datasets/wang_4mice_john/clips/OFTsocial5mice-0000-00.mp4.old.predictions@3700-6260.mp4", "dataset": "mice_of", "n_instances": 5},
    {"test_data_path": "sleap-data/datasets/BermanFlies/clips/072212_163153@10000-13200.mp4", "dataset": "fly32", "n_instances": 1},
    {"test_data_path": "sleap-data/datasets/wt_gold.13pt/clips/single_fly@100000-110000.mp4", "dataset": "flies13", "n_instances": 1},
    {"test_data_path": "sleap-data/datasets/wt_gold.13pt/clips/190719_090330_wt_18159206_rig1.2@15000-17560.mp4", "dataset": "flies13", "n_instances": 2},
    {"test_data_path": "sleap-data/datasets/wt_gold.13pt/clips/three_flies@8800-16800.mp4", "dataset": "flies13", "n_instances": 3},
    {"test_data_path": "sleap-data/datasets/wt_gold.13pt/clips/four_flies@1000-11000.mp4", "dataset": "flies13", "n_instances": 4},
    {"test_data_path": "sleap-data/datasets/wt_gold.13pt/clips/eight_flies@180000-200000.mp4", "dataset": "flies13", "n_instances": 8},
    {"test_data_path": "sleap-data/datasets/nyu-gerbils/clips/2020-3-10_daytime_5mins_compressedTalmo@3200-5760.mp4", "dataset": "gerbils", "n_instances": 4},
])

In [ ]:
precisions = [16, 32]
batch_sizes = [1, 4, 8, 16, 32]
n_frames = 1280
reps = 5

model, clips = mice_of_td, mice_of_clips
# model, clips = flies13_td, flies13_clips
# model, clips = flies13_td_id, flies13_clips
# model, clips = fly32_single, fly32_clips
# model, clips = gerbils_td_id, gerbils_clips

for test_data in clips:
    for precision in precisions:
        for batch_size in batch_sizes:
            !python benchmark.py "{model}" {precision} "{test_data}" --n_frames {n_frames} --batch_size {batch_size} --reps {reps}

In [ ]:
precisions = [16, 32]
batch_sizes = [1, 4, 8, 16, 32]
n_frames = 1280
reps = 5


# model, clips = mice_of_td, mice_of_clips
model, clips = flies13_td, flies13_clips
# model, clips = flies13_td_id, flies13_clips
# model, clips = fly32_single, fly32_clips
# model, clips = gerbils_td_id, gerbils_clips

for test_data in clips:
    for precision in precisions:
        for batch_size in batch_sizes:
            !python benchmark.py "{model}" {precision} "{test_data}" --n_frames {n_frames} --batch_size {batch_size} --reps {reps}

In [ ]:
precisions = [16, 32]
batch_sizes = [1, 4, 8, 16, 32]
n_frames = 1280
reps = 5


# model, clips = mice_of_td, mice_of_clips
# model, clips = flies13_td, flies13_clips
model, clips = flies13_td_id, flies13_clips
# model, clips = fly32_single, fly32_clips
# model, clips = gerbils_td_id, gerbils_clips

for test_data in clips:
    for precision in precisions:
        for batch_size in batch_sizes:
            !python benchmark.py "{model}" {precision} "{test_data}" --n_frames {n_frames} --batch_size {batch_size} --reps {reps}

In [ ]:
precisions = [16, 32]
batch_sizes = [1, 4, 8, 16, 32, 64, 128, 256]
n_frames = 1280
reps = 5


# model, clips = mice_of_td, mice_of_clips
# model, clips = flies13_td, flies13_clips
# model, clips = flies13_td_id, flies13_clips
model, clips = fly32_single, fly32_clips
# model, clips = gerbils_td_id, gerbils_clips

for test_data in clips:
    for precision in precisions:
        for batch_size in batch_sizes:
            !python benchmark.py "{model}" {precision} "{test_data}" --n_frames {n_frames} --batch_size {batch_size} --reps {reps}

In [ ]:
precisions = [16, 32]
batch_sizes = [1, 4, 8, 16, 32]
n_frames = 1280
reps = 5


# model, clips = mice_of_td, mice_of_clips
# model, clips = flies13_td, flies13_clips
# model, clips = flies13_td_id, flies13_clips
# model, clips = fly32_single, fly32_clips
model, clips = gerbils_td_id, gerbils_clips

for test_data in clips:
    for precision in precisions:
        for batch_size in batch_sizes:
            !python benchmark.py "{model}" {precision} "{test_data}" --n_frames {n_frames} --batch_size {batch_size} --reps {reps} --no-grayscale

In [ ]:
all_benchmarks = pd.read_csv("trt_benchmarks.csv")
all_benchmarks = all_benchmarks.merge(test_data_info, on="test_data_path")
all_benchmarks.to_csv("all_trt_benchmarks.csv", index=False)

all_benchmarks.groupby(["dataset", "n_instances", "batch_size"])[["fps"]].agg(["mean", "std"])